In [3]:
import tensorflow as tf
tf.__version__
tf.config.experimental.enable_op_determinism()

In [4]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_absolute_error
import random

In [5]:
subject = 'Espírito Santo - Consumo de Cimento (t)'
split_index = 17 #Referente aos 18 anos de input  

In [6]:
data = pd.read_csv('2003_01_model_input_ES.csv')
data = data[[col for col in data.columns if col != subject] + [subject]] #Seta consumo (target) para a coluna final
data =data.drop([' NFSP - Fluxo Mensal (Milhões de reais)'], axis=1)
data

,Ano,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,...,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - PIB - Estadual.1,Espírito Santo - PIB - Construção Civil.1,Espírito Santo - PIB - Per Capita.1,Espírito Santo - PIB - Preços de Mercado.1,Espírito Santo - Consumo de Cimento (t).1,Espírito Santo - Consumo de Cimento (t)
0,2003,0.760059,3.282770,8.177444,1197.520265,0.651697,9.199031,20.905641,10.949017,1.772069e+08,...,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,635.153000,635.153000
1,2004,0.762013,3.237874,8.107574,1241.084984,0.563637,5.770455,15.098325,7.757945,1.793848e+08,...,6.442102e+07,3.565093e+06,16.896386,5.992647e+07,6.442102e+07,3.565093e+06,16.896386,5.992647e+07,636.659000,636.659000
2,2005,0.763817,3.103762,8.084884,1360.910937,0.440257,4.275691,17.415318,5.579973,1.814674e+08,...,6.528363e+07,3.601258e+06,16.957151,6.021671e+07,6.528363e+07,3.601258e+06,16.957151,6.021671e+07,768.967000,768.967000
3,2006,0.765477,3.130953,7.998040,1619.896813,0.416613,3.830878,14.056528,6.977991,1.834544e+08,...,6.602371e+07,3.639069e+06,16.990689,6.039472e+07,6.602371e+07,3.639069e+06,16.990689,6.039472e+07,673.561000,673.561000
4,2007,0.766998,3.335837,7.969140,1900.181823,0.424627,3.564317,11.230192,6.441723,1.853458e+08,...,6.663355e+07,3.646642e+06,16.999308,6.046495e+07,6.663355e+07,3.646642e+06,16.999308,6.046495e+07,974.754000,974.754000
5,2008,0.768383,3.594891,7.934578,2065.641254,0.415443,2.868108,11.760983,5.071124,1.871410e+08,...,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,1263.713000,1263.713000
6,2009,0.769638,3.774692,7.964300,2200.127740,0.446751,2.305885,9.472585,6.040326,1.888400e+08,...,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,1138.824000,1138.824000
7,2010,0.770767,3.930070,7.927576,2376.087362,0.464446,1.958246,9.332485,6.541237,1.907558e+08,...,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,1223.466000,1223.466000
8,2011,0.771775,4.104068,7.917310,2497.788723,0.468365,2.079088,10.985689,6.090625,1.919894e+08,...,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,1271.961000,1271.961000
9,2012,0.772417,4.250916,9.395868,2521.227215,0.489965,2.183282,8.159014,5.984749,1.932582e+08,...,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,1354.740000,1354.740000


In [7]:
input_data = data.iloc[:split_index + 1, 1:-1]
input_data

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,INCC (%).1,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - PIB - Estadual.1,Espírito Santo - PIB - Construção Civil.1,Espírito Santo - PIB - Per Capita.1,Espírito Santo - PIB - Preços de Mercado.1,Espírito Santo - Consumo de Cimento (t).1
0,0.760059,3.282770,8.177444,1197.520265,0.651697,9.199031,20.905641,10.949017,1.772069e+08,7.330309e+06,...,0.895303,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,6.342537e+07,3.523987e+06,16.799308,5.948249e+07,635.153000
1,0.762013,3.237874,8.107574,1241.084984,0.563637,5.770455,15.098325,7.757945,1.793848e+08,7.397521e+06,...,0.746570,6.442102e+07,3.565093e+06,16.896386,5.992647e+07,6.442102e+07,3.565093e+06,16.896386,5.992647e+07,636.659000
2,0.763817,3.103762,8.084884,1360.910937,0.440257,4.275691,17.415318,5.579973,1.814674e+08,7.462916e+06,...,0.589811,6.528363e+07,3.601258e+06,16.957151,6.021671e+07,6.528363e+07,3.601258e+06,16.957151,6.021671e+07,768.967000
3,0.765477,3.130953,7.998040,1619.896813,0.416613,3.830878,14.056528,6.977991,1.834544e+08,7.526672e+06,...,0.606044,6.602371e+07,3.639069e+06,16.990689,6.039472e+07,6.602371e+07,3.639069e+06,16.990689,6.039472e+07,673.561000
4,0.766998,3.335837,7.969140,1900.181823,0.424627,3.564317,11.230192,6.441723,1.853458e+08,7.589168e+06,...,0.633401,6.663355e+07,3.646642e+06,16.999308,6.046495e+07,6.663355e+07,3.646642e+06,16.999308,6.046495e+07,974.754000
5,0.768383,3.594891,7.934578,2065.641254,0.415443,2.868108,11.760983,5.071124,1.871410e+08,7.647774e+06,...,0.587619,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,6.711793e+07,3.666068e+06,16.994002,6.044984e+07,1263.713000
6,0.769638,3.774692,7.964300,2200.127740,0.446751,2.305885,9.472585,6.040326,1.888400e+08,7.700574e+06,...,0.566759,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,6.738295e+07,3.683277e+06,16.935445,6.029388e+07,1138.824000
7,0.770767,3.930070,7.927576,2376.087362,0.464446,1.958246,9.332485,6.541237,1.907558e+08,7.747985e+06,...,0.593537,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,6.901541e+07,3.673390e+06,16.970783,6.069300e+07,1223.466000
8,0.771775,4.104068,7.917310,2497.788723,0.468365,2.079088,10.985689,6.090625,1.919894e+08,7.786622e+06,...,0.600821,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,7.032342e+07,3.655681e+06,16.962954,6.083140e+07,1271.961000
9,0.772417,4.250916,9.395868,2521.227215,0.489965,2.183282,8.159014,5.984749,1.932582e+08,7.815135e+06,...,0.604731,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,7.118644e+07,3.633845e+06,16.845395,6.065703e+07,1354.740000


In [8]:
# Shift para prever futuro e não presente
target_data = data[subject].shift(-1)
target_data

0      636.659000
1      768.967000
2      673.561000
3      974.754000
4     1263.713000
5     1138.824000
6     1223.466000
7     1271.961000
8     1354.740000
9     1378.303000
10    1296.041857
11    1250.323886
12    1210.917257
13    1097.369000
14    1146.390000
15    1139.906000
16    1138.655000
17    1135.022000
18    1040.260000
19            NaN
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [9]:
# input para treinamento
train_input = (input_data.iloc[:-1] - np.mean(input_data.iloc[:-1], axis=0)) / np.std(input_data.iloc[:-1], axis=0)
train_input

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,INCC (%).1,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - PIB - Estadual.1,Espírito Santo - PIB - Construção Civil.1,Espírito Santo - PIB - Per Capita.1,Espírito Santo - PIB - Preços de Mercado.1,Espírito Santo - Consumo de Cimento (t).1
0,-1.890494,-1.225902,-0.764918,-1.095771,2.057317,3.335319,2.503268,1.984588,-1.915550,-2.217697,...,2.670619,-1.833535,-0.888456,0.123092,-1.631909,-1.833535,-0.888456,0.123092,-1.631909,-1.833992
1,-1.556140,-1.310067,-0.822986,-1.014041,1.041262,1.436308,0.974434,0.428265,-1.603622,-1.796936,...,1.398420,-1.520319,-0.282403,0.596678,-0.539076,-1.520319,-0.282403,0.596678,-0.539076,-1.827865
2,-1.247300,-1.561483,-0.841843,-0.789240,-0.382310,0.608392,1.584406,-0.633957,-1.305339,-1.387550,...,0.057571,-1.248956,0.250792,0.893115,0.175364,-1.248956,0.250792,0.893115,0.175364,-1.289585
3,-0.963178,-1.510509,-0.914016,-0.303366,-0.655123,0.362019,0.700171,0.047872,-1.020736,-0.988421,...,0.196419,-1.016136,0.808255,1.056729,0.613532,-1.016136,0.808255,1.056729,0.613532,-1.677733
4,-0.702973,-1.126418,-0.938034,0.222467,-0.562655,0.214378,-0.043890,-0.213672,-0.749851,-0.597180,...,0.430422,-0.824291,0.919918,1.098773,0.786398,-0.824291,0.919918,1.098773,0.786398,-0.452364
5,-0.465887,-0.640777,-0.966757,0.532880,-0.668618,-0.171237,0.095846,-0.882129,-0.492719,-0.230290,...,0.038818,-0.671911,1.206320,1.072888,0.749189,-0.671911,1.206320,1.072888,0.749189,0.723233
6,-0.251122,-0.303710,-0.942056,0.785185,-0.307381,-0.482640,-0.506598,-0.409438,-0.249377,0.100246,...,-0.139605,-0.588541,1.460043,0.787225,0.365309,-0.588541,1.460043,0.787225,0.365309,0.215136
7,-0.057878,-0.012427,-0.972576,1.115296,-0.103220,-0.675189,-0.543480,-0.165137,0.025012,0.397049,...,0.089437,-0.074993,1.314280,0.959620,1.347726,-0.074993,1.314280,0.959620,1.347726,0.559492
8,0.114644,0.313763,-0.981108,1.343616,-0.057995,-0.608258,-0.108258,-0.384906,0.201698,0.638927,...,0.151742,0.336487,1.053173,0.921424,1.688405,0.336487,1.053173,0.921424,1.688405,0.756789
9,0.224627,0.589055,0.247678,1.387588,0.191223,-0.550547,-0.852408,-0.436543,0.383423,0.817429,...,0.185184,0.607980,0.731237,0.347923,1.259187,0.607980,0.731237,0.347923,1.259187,1.093566


In [10]:
# Alvo para treinamento
train_target = target_data.iloc[:split_index]
train_target

0      636.659000
1      768.967000
2      673.561000
3      974.754000
4     1263.713000
5     1138.824000
6     1223.466000
7     1271.961000
8     1354.740000
9     1378.303000
10    1296.041857
11    1250.323886
12    1210.917257
13    1097.369000
14    1146.390000
15    1139.906000
16    1138.655000
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [11]:
#input de test (Ano 2021)
test_input = ((input_data - np.mean(input_data, axis=0)) / np.std(input_data, axis=0)).iloc[-1:]
test_input

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,INCC (%).1,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - PIB - Estadual.1,Espírito Santo - PIB - Construção Civil.1,Espírito Santo - PIB - Per Capita.1,Espírito Santo - PIB - Preços de Mercado.1,Espírito Santo - Consumo de Cimento (t).1
17,0.910005,2.096358,0.751123,-1.604501,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,...,1.568753,0.700733,-0.492713,-1.07591,-1.001567,0.700733,-0.492713,-1.07591,-1.001567,0.208141


In [12]:
# Alvo de test (Ano 2021)
test_target = target_data.iloc[split_index:split_index + 1]
test_target

17    1135.022
Name: Espírito Santo - Consumo de Cimento (t), dtype: float64

In [13]:
# Rede neural feed-forward com optmizador Estocástico
def neural_network_model(train_input, train_target, want_verbose=1, seed=0):
    if seed != 0:
        random.seed(seed)
        np.random.seed(seed)
        tf.random.set_seed(seed)
    # Aṕos 500 epochs sem grandes melhoras no val_loss, interrompe.
    early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                                      patience=500, 
                                                      verbose=want_verbose, 
                                                      restore_best_weights=True,
                                                      start_from_epoch=500)
    # Método estocástico e learning rate=0.005
    optimizer = tf.keras.optimizers.SGD(learning_rate=0.005)
    model = tf.keras.Sequential([
        tf.keras.layers.Dense(2048, activation='tanh', input_shape=(train_input.shape[1],)),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dropout(0.3),
        tf.keras.layers.Dense(1024, activation='tanh'),
        tf.keras.layers.Dense(128, activation='tanh'),
        tf.keras.layers.Dense(1)
    ])
    validation_size = len(train_input)//5
    model.compile(optimizer=optimizer, loss='mean_squared_error')   
    history = model.fit(train_input, 
                      train_target, 
                      epochs=10000,
                      validation_data=(train_input.iloc[-validation_size:], 
                                       train_target.iloc[-validation_size:]),
                      callbacks=[early_stopping], 
                      verbose=want_verbose)
    return model, history

In [14]:
def get_a_good_seed(train_input, train_target, test_target, test_input):

    random_seeds = [random.randint(0, 2**32 - 1) for _ in range(100)]
    print(random_seeds)

    best_loss = float('inf')
    winner_seed = None
    i = 0
    for seed in random_seeds:
        print(f"\n\nStep: {i} ___________________________________________")
        i += 1

        model, history = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)
        current_loss = min(history.history['val_loss'][500:])
        print(f"val_loss: {current_loss}")

        if current_loss < best_loss:
            best_loss = current_loss
            winner_seed = seed
            print(f"winner_seed: {winner_seed}")
            if winner_seed == 0.0:
                return winner_seed

    return winner_seed

In [16]:
winner_seed = get_a_good_seed(train_input, train_target, test_target, test_input)
print(f"\n\nfinal_seed: {winner_seed}")

[3935459643, 43344088, 2754262436, 4144423298, 3696462778, 707852907, 2561258774, 2935066208, 1272860112, 51788887, 1838750500, 2283018507, 2423224811, 319724855, 3305490446, 1896380322, 1548417577, 1841342927, 45154810, 207712292, 2770651927, 293255723, 311452532, 1880160673, 3908204544, 897702259, 1473933256, 1334123459, 2216910919, 3860192771, 3512556535, 3603086982, 2104534144, 3897491492, 3349715790, 1709954608, 514405191, 117257054, 1705578560, 1716559869, 3686346975, 1066198456, 721585716, 1309624248, 1166309498, 2496747790, 277973490, 2504436584, 1998957133, 205802174, 4237694886, 3814938971, 628739943, 3489702438, 2485665449, 149517640, 556563461, 1611663053, 1798139692, 4061544113, 3860744869, 2717314086, 2677155790, 879093296, 1725279258, 253522399, 1411619490, 1291683912, 38793310, 3537616811, 1134152629, 3821667631, 468390055, 3748150296, 1570654986, 2927130423, 730537319, 1375152961, 3624726737, 1604893062, 2324007491, 2736724683, 3088668042, 3230544226, 738622624, 408632

val_loss: 11.493034362792969


Step: 84 ___________________________________________
val_loss: 7.689043045043945


Step: 85 ___________________________________________
val_loss: 11.691482543945312


Step: 86 ___________________________________________
val_loss: 9.33243465423584


Step: 87 ___________________________________________
val_loss: 11.494023323059082


Step: 88 ___________________________________________
val_loss: 11.550064086914062


Step: 89 ___________________________________________
val_loss: 457.3650817871094


Step: 90 ___________________________________________
val_loss: 11.493071556091309


Step: 91 ___________________________________________
val_loss: 12.039656639099121


Step: 92 ___________________________________________
val_loss: 12.11919116973877


Step: 93 ___________________________________________
val_loss: 65.78227233886719


Step: 94 ___________________________________________
val_loss: 11.506577491760254


Step: 95 __________________________________________

In [17]:
trained_model, history = neural_network_model(train_input, train_target, want_verbose=1, seed=winner_seed)

Epoch 1/10000
1/1 [==============================] - 0s 438ms/step - loss: 1292027.0000 - val_loss: 1221197.3750
Epoch 2/10000
1/1 [==============================] - 0s 14ms/step - loss: 1221177.8750 - val_loss: 151984.5781
Epoch 3/10000
1/1 [==============================] - 0s 14ms/step - loss: 1070838.1250 - val_loss: 1056153.1250
Epoch 4/10000
1/1 [==============================] - 0s 14ms/step - loss: 1111257.3750 - val_loss: 2207608.0000
Epoch 5/10000
1/1 [==============================] - 0s 14ms/step - loss: 1109430.1250 - val_loss: 2158508.0000
Epoch 6/10000
1/1 [==============================] - 0s 14ms/step - loss: 1174527.0000 - val_loss: 1588711.3750
Epoch 7/10000
1/1 [==============================] - 0s 14ms/step - loss: 1048116.1250 - val_loss: 1798103.0000
Epoch 8/10000
1/1 [==============================] - 0s 13ms/step - loss: 995204.0000 - val_loss: 1868200.1250
Epoch 9/10000
1/1 [==============================] - 0s 13ms/step - loss: 1006328.9375 - val_loss: 181074

1/1 [==============================] - 0s 13ms/step - loss: 115473.9062 - val_loss: 108368.8438
Epoch 148/10000
1/1 [==============================] - 0s 14ms/step - loss: 116199.8984 - val_loss: 120966.5703
Epoch 149/10000
1/1 [==============================] - 0s 13ms/step - loss: 110098.4922 - val_loss: 111487.6016
Epoch 150/10000
1/1 [==============================] - 0s 14ms/step - loss: 108275.4766 - val_loss: 117016.8359
Epoch 151/10000
1/1 [==============================] - 0s 13ms/step - loss: 105745.9609 - val_loss: 116876.1641
Epoch 152/10000
1/1 [==============================] - 0s 13ms/step - loss: 108462.1875 - val_loss: 103129.6797
Epoch 153/10000
1/1 [==============================] - 0s 13ms/step - loss: 104075.6250 - val_loss: 104565.3672
Epoch 154/10000
1/1 [==============================] - 0s 14ms/step - loss: 110848.3594 - val_loss: 92683.1484
Epoch 155/10000
1/1 [==============================] - 0s 13ms/step - loss: 100482.3125 - val_loss: 105452.0000
Epoch 156

1/1 [==============================] - 0s 14ms/step - loss: 35241.9180 - val_loss: 7721.5566
Epoch 296/10000
1/1 [==============================] - 0s 13ms/step - loss: 38094.4297 - val_loss: 5445.5630
Epoch 297/10000
1/1 [==============================] - 0s 13ms/step - loss: 38142.7891 - val_loss: 6093.8687
Epoch 298/10000
1/1 [==============================] - 0s 13ms/step - loss: 35022.8164 - val_loss: 7447.7480
Epoch 299/10000
1/1 [==============================] - 0s 14ms/step - loss: 35957.0000 - val_loss: 7395.1875
Epoch 300/10000
1/1 [==============================] - 0s 14ms/step - loss: 37151.2812 - val_loss: 5220.5137
Epoch 301/10000
1/1 [==============================] - 0s 13ms/step - loss: 34838.3008 - val_loss: 6901.8237
Epoch 302/10000
1/1 [==============================] - 0s 14ms/step - loss: 36729.8945 - val_loss: 7751.7695
Epoch 303/10000
1/1 [==============================] - 0s 13ms/step - loss: 35107.8242 - val_loss: 6930.6108
Epoch 304/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 29782.5508 - val_loss: 626.6970
Epoch 446/10000
1/1 [==============================] - 0s 13ms/step - loss: 29446.0371 - val_loss: 604.6086
Epoch 447/10000
1/1 [==============================] - 0s 13ms/step - loss: 26397.9062 - val_loss: 1151.8798
Epoch 448/10000
1/1 [==============================] - 0s 13ms/step - loss: 30005.3555 - val_loss: 574.3656
Epoch 449/10000
1/1 [==============================] - 0s 13ms/step - loss: 29095.2266 - val_loss: 387.4405
Epoch 450/10000
1/1 [==============================] - 0s 13ms/step - loss: 29740.3359 - val_loss: 546.6069
Epoch 451/10000
1/1 [==============================] - 0s 13ms/step - loss: 30050.7344 - val_loss: 541.8903
Epoch 452/10000
1/1 [==============================] - 0s 13ms/step - loss: 32847.6875 - val_loss: 173.0630
Epoch 453/10000
1/1 [==============================] - 0s 14ms/step - loss: 29674.5020 - val_loss: 373.7822
Epoch 454/10000
1/1 [======================

Epoch 521/10000
1/1 [==============================] - 0s 13ms/step - loss: 21032.0684 - val_loss: 1563.0453
Epoch 522/10000
1/1 [==============================] - 0s 14ms/step - loss: 15282.0947 - val_loss: 1927.9059
Epoch 523/10000
1/1 [==============================] - 0s 13ms/step - loss: 18926.4023 - val_loss: 1583.2406
Epoch 524/10000
1/1 [==============================] - 0s 13ms/step - loss: 15569.3477 - val_loss: 1468.6195
Epoch 525/10000
1/1 [==============================] - 0s 13ms/step - loss: 17088.1562 - val_loss: 1507.7523
Epoch 526/10000
1/1 [==============================] - 0s 13ms/step - loss: 18604.1816 - val_loss: 1612.8024
Epoch 527/10000
1/1 [==============================] - 0s 13ms/step - loss: 16952.0645 - val_loss: 22075.5293
Epoch 528/10000
1/1 [==============================] - 0s 13ms/step - loss: 34186.0586 - val_loss: 1624.9064
Epoch 529/10000
1/1 [==============================] - 0s 13ms/step - loss: 23290.8555 - val_loss: 1639.9327
Epoch 530/10000
1/

...

Epoch 747/10000
1/1 [==============================] - 0s 18ms/step - loss: 23632.4727 - val_loss: 11.7126
Epoch 748/10000
1/1 [==============================] - 0s 13ms/step - loss: 20162.0293 - val_loss: 55.1005
Epoch 749/10000
1/1 [==============================] - 0s 13ms/step - loss: 16370.9609 - val_loss: 254.0557
Epoch 750/10000
1/1 [==============================] - 0s 13ms/step - loss: 12918.8242 - val_loss: 97.8833
Epoch 751/10000
1/1 [==============================] - 0s 14ms/step - loss: 21911.8828 - val_loss: 12.4903
Epoch 752/10000
1/1 [==============================] - 0s 13ms/step - loss: 14074.1348 - val_loss: 73.3735
Epoch 753/10000
1/1 [==============================] - 0s 13ms/step - loss: 13482.3848 - val_loss: 84.7043
Epoch 754/10000
1/1 [==============================] - 0s 13ms/step - loss: 21229.1074 - val_loss: 24.0511
Epoch 755/10000
1/1 [==============================] - 0s 13ms/step - loss: 13468.1172 - val_loss: 718.7898
Epoch 756/10000
1/1 [==============

1/1 [==============================] - 0s 13ms/step - loss: 36379.3008 - val_loss: 144.7736
Epoch 900/10000
1/1 [==============================] - 0s 14ms/step - loss: 29956.8867 - val_loss: 15.8471
Epoch 901/10000
1/1 [==============================] - 0s 13ms/step - loss: 30191.3457 - val_loss: 15.1369
Epoch 902/10000
1/1 [==============================] - 0s 13ms/step - loss: 29925.1523 - val_loss: 19.9658
Epoch 903/10000
1/1 [==============================] - 0s 13ms/step - loss: 29874.7969 - val_loss: 15.1592
Epoch 904/10000
1/1 [==============================] - 0s 13ms/step - loss: 34493.1602 - val_loss: 102.8302
Epoch 905/10000
1/1 [==============================] - 0s 13ms/step - loss: 29873.3926 - val_loss: 11.7490
Epoch 906/10000
1/1 [==============================] - 0s 13ms/step - loss: 26858.2168 - val_loss: 238.7864
Epoch 907/10000
1/1 [==============================] - 0s 13ms/step - loss: 30068.9883 - val_loss: 29.7431
Epoch 908/10000
1/1 [=============================

1/1 [==============================] - 0s 14ms/step - loss: 23772.6855 - val_loss: 538.0346
Epoch 1051/10000
1/1 [==============================] - 0s 14ms/step - loss: 21420.7422 - val_loss: 4930.0298
Epoch 1052/10000
1/1 [==============================] - 0s 14ms/step - loss: 20715.2578 - val_loss: 946.8333
Epoch 1053/10000
1/1 [==============================] - 0s 13ms/step - loss: 19826.3906 - val_loss: 1971.5835
Epoch 1054/10000
1/1 [==============================] - 0s 14ms/step - loss: 20138.4180 - val_loss: 963.0534
Epoch 1055/10000
1/1 [==============================] - 0s 14ms/step - loss: 22819.9004 - val_loss: 844.8968
Epoch 1056/10000
1/1 [==============================] - 0s 14ms/step - loss: 21741.8555 - val_loss: 1106.3231
Epoch 1057/10000
1/1 [==============================] - 0s 14ms/step - loss: 18439.8750 - val_loss: 1192.7029
Epoch 1058/10000
1/1 [==============================] - 0s 13ms/step - loss: 18038.5020 - val_loss: 3353.3503
Epoch 1059/10000
1/1 [=========

Epoch 1125/10000
1/1 [==============================] - 0s 13ms/step - loss: 12031.6191 - val_loss: 2965.6614
Epoch 1126/10000
1/1 [==============================] - 0s 14ms/step - loss: 10995.6260 - val_loss: 1448.9395
Epoch 1127/10000
1/1 [==============================] - 0s 13ms/step - loss: 10945.8447 - val_loss: 10525.9531
Epoch 1128/10000
1/1 [==============================] - 0s 13ms/step - loss: 9747.6035 - val_loss: 11604.5049
Epoch 1129/10000
1/1 [==============================] - 0s 14ms/step - loss: 9024.9785 - val_loss: 7720.6890
Epoch 1130/10000
1/1 [==============================] - 0s 13ms/step - loss: 8722.9004 - val_loss: 10713.9092
Epoch 1131/10000
1/1 [==============================] - 0s 13ms/step - loss: 10938.8691 - val_loss: 5631.7339
Epoch 1132/10000
1/1 [==============================] - 0s 13ms/step - loss: 10964.8262 - val_loss: 8530.5420
Epoch 1133/10000
1/1 [==============================] - 0s 13ms/step - loss: 9792.9873 - val_loss: 12305.1289
Epoch 1134

Epoch 1200/10000
1/1 [==============================] - 0s 13ms/step - loss: 26049.8477 - val_loss: 2462.4934
Epoch 1201/10000
1/1 [==============================] - 0s 14ms/step - loss: 25782.2734 - val_loss: 2628.8240
Epoch 1202/10000
1/1 [==============================] - 0s 13ms/step - loss: 25727.0801 - val_loss: 2407.6199
Epoch 1203/10000
1/1 [==============================] - 0s 14ms/step - loss: 22695.7324 - val_loss: 3697.2102
Epoch 1204/10000
1/1 [==============================] - 0s 14ms/step - loss: 24005.1348 - val_loss: 1950.0541
Epoch 1205/10000
1/1 [==============================] - 0s 13ms/step - loss: 26397.4160 - val_loss: 1651.4200
Epoch 1206/10000
1/1 [==============================] - 0s 14ms/step - loss: 26085.4668 - val_loss: 3666.4375
Epoch 1207/10000
1/1 [==============================] - 0s 13ms/step - loss: 37976.9141 - val_loss: 680.4550
Epoch 1208/10000
1/1 [==============================] - 0s 13ms/step - loss: 30899.1211 - val_loss: 13.8371
Epoch 1209/10

In [18]:
prediction = trained_model.predict(test_input)
print(f"{test_target.values}(test_target) - {prediction}(prediction) = {mean_absolute_error(test_target, prediction)}")

1/1 [==============================] - 0s 47ms/step
[1135.022](test_target) - [[1144.1735]](prediction) = 9.151461914062565


In [19]:
def model_time_series_mae_tester(data, window, seed):
    if window > 2:

        # Copia e organiza data geral
        predictions = []
        targets = []
        data_copy = data.copy()
        data_copy[subject] = data[subject].shift(-1)
        data_copy = data_copy.iloc[:-1,1:]
        
        input_df = data_copy.iloc[:, :-1]
        target_df = data_copy.iloc[:, -1:]

        for i in range(window, len(data_copy) + 1):

            train_input = input_df.iloc[:i - 1]
            train_input = (train_input - np.mean(train_input, axis=0)) / np.std(train_input, axis=0)
            print("train_input:")
            display(train_input)

            train_target = target_df.iloc[:i - 1]
            print("train_target:")
            display(train_target)

            test_input = input_df.iloc[:i]
            test_input = ((test_input - np.mean(test_input, axis=0)) / np.std(test_input, axis=0)).iloc[-1:]
            print("test_input:")
            display(test_input)

            test_target = target_df.iloc[i - 1:i]
            print("test_target:")
            display(test_target)
            
            model, hitory = neural_network_model(train_input, train_target, want_verbose=0, seed=seed)

            prediction = model.predict(test_input)

            current_error = mean_absolute_error(test_target, prediction)
            print(f"Error: {current_error}\n\n")

            predictions.append(prediction[0].item())
            targets.append(test_target.iloc[0, 0])

        return predictions, targets

In [20]:
predictions, targets = model_time_series_mae_tester(data.iloc[:-1,:], 9, winner_seed)
predictions

train_input:


,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,INCC (%).1,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - PIB - Estadual.1,Espírito Santo - PIB - Construção Civil.1,Espírito Santo - PIB - Per Capita.1,Espírito Santo - PIB - Preços de Mercado.1,Espírito Santo - Consumo de Cimento (t).1
0,-1.660580,-0.491540,1.833233,-1.286771,2.184461,2.272299,1.914650,2.360960,-1.590751,-1.593283,...,2.299346,-1.647839,-1.904113,-2.273856,-2.143985,-1.647839,-1.904113,-2.273856,-2.143985,-1.110489
1,-1.104592,-0.647955,1.017393,-1.184412,1.077411,0.707092,0.380286,0.491065,-1.095308,-1.106645,...,0.891536,-1.048525,-1.128084,-0.736403,-0.887827,-1.048525,-1.128084,-0.736403,-0.887827,-1.104499
2,-0.591032,-1.115197,0.752453,-0.902872,-0.473651,0.024704,0.992464,-0.785177,-0.621537,-0.633164,...,-0.592240,-0.529294,-0.445347,0.225948,-0.066614,-0.529294,-0.445347,0.225948,-0.066614,-0.578323
3,-0.118573,-1.020465,-0.261576,-0.294365,-0.770896,-0.178361,0.105030,0.034030,-0.169496,-0.171546,...,-0.438592,-0.083809,0.268464,0.757106,0.437039,-0.083809,0.268464,0.757106,0.437039,-0.957743
4,0.314114,-0.306654,-0.599026,0.364185,-0.670147,-0.300051,-0.641723,-0.280211,0.260758,0.280950,...,-0.179645,0.283271,0.411445,0.893598,0.635740,0.283271,0.411445,0.893598,0.635740,0.240074
5,0.708357,0.595880,-1.002589,0.752944,-0.785600,-0.617883,-0.501481,-1.083350,0.669167,0.705281,...,-0.612992,0.574838,0.778171,0.809566,0.592970,0.574838,0.778171,0.809566,0.592970,1.389237
6,1.065484,1.222300,-0.655538,1.068930,-0.392011,-0.874548,-1.106104,-0.515420,1.055674,1.087568,...,-0.810435,0.734361,1.103054,-0.117810,0.151718,0.734361,1.103054,-0.117810,0.151718,0.892565
7,1.386823,1.763633,-1.084349,1.482361,-0.169566,-1.033252,-1.143120,-0.221898,1.491492,1.430840,...,-0.556978,1.716996,0.916410,0.441852,1.280959,1.716996,0.916410,0.441852,1.280959,1.229178


train_target:


,Espírito Santo - Consumo de Cimento (t)
0,636.659
1,768.967
2,673.561
3,974.754
4,1263.713
5,1138.824
6,1223.466
7,1271.961


test_input:


...

,Espírito Santo - IDH,Espírito Santo - Valor Cimento Portland,Espírito Santo - Desemprego,Espírito Santo - Produção de Cimento (t),IPCA - Variação mensal durante o Plano Real (%),NFSP - Porcentagem do PIB (%),Taxa Selic (%),IGP-DI,População,Estoque liquido de capital fixo - (R$),...,INCC (%).1,Espírito Santo - PIB - Estadual,Espírito Santo - PIB - Construção Civil,Espírito Santo - PIB - Per Capita,Espírito Santo - PIB - Preços de Mercado,Espírito Santo - PIB - Estadual.1,Espírito Santo - PIB - Construção Civil.1,Espírito Santo - PIB - Per Capita.1,Espírito Santo - PIB - Preços de Mercado.1,Espírito Santo - Consumo de Cimento (t).1
17,0.910005,2.096358,0.751123,-1.604501,0.903737,-0.736617,-1.952958,2.793475,1.369386,0.051931,...,1.568753,0.700733,-0.492713,-1.07591,-1.001567,0.700733,-0.492713,-1.07591,-1.001567,0.208141


test_target:


,Espírito Santo - Consumo de Cimento (t)
17,1135.022


1/1 [==============================] - 0s 34ms/step
Error: 9.151461914062565




[1242.609130859375,
 1250.5382080078125,
 1190.31640625,
 1058.276611328125,
 1244.2540283203125,
 1267.0032958984375,
 1154.1085205078125,
 1150.619873046875,
 1117.8826904296875,
 1144.1734619140625]

In [21]:
display(targets)
display(predictions)

[1354.74,
 1378.303,
 1296.041857142857,
 1250.3238857142856,
 1210.9172571428571,
 1097.369,
 1146.39,
 1139.906,
 1138.655,
 1135.022]

[1242.609130859375,
 1250.5382080078125,
 1190.31640625,
 1058.276611328125,
 1244.2540283203125,
 1267.0032958984375,
 1154.1085205078125,
 1150.619873046875,
 1117.8826904296875,
 1144.1734619140625]

In [22]:
mae = mean_absolute_error(predictions, targets)
mae

78.89956185267856

In [23]:
porcentage = mae/np.mean(targets)
porcentage

0.06495037718571052